<a href="https://colab.research.google.com/github/kobrue02/evaluating-llm-generated-nlu-data/blob/main/bin/notebooks/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the repository and cd into it

In [ ]:
!git clone https://github.com/kobrue02/evaluating-llm-generated-nlu-data/
%cd evaluating-llm-generated-nlu-data
!git pull

Import the necessary classes and methods

In [2]:
from bin.data_generation.generate_data import DataGenerationModel
from bin.data_generation.construct_prompt import load_prompt
from bin.utils.types import DataSet
from transformers import AutoModelForCausalLM, AutoTokenizer

Initialize Phi-3.5 Model

In [ ]:
phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Load the model

In [4]:
phi = DataGenerationModel(model=phi_model, tokenizer=phi_tokenizer)

Generate data with basic prompt

In [ ]:
from bin.data_generation.consts import INTENTS, PROMPT_IDS, NUM_SAMPLES
for prompt_id in PROMPT_IDS:
  dataset = phi.build_dataset_from_intents(prompt_id, INTENTS, NUM_SAMPLES)
  dataset.to_csv(f"{prompt_id}_data.csv", columns=["query", "intent"])
  dataset.to_data_frame()
